# 0 - Requirements

In [ ]:
import requests
import json
import os
from requests.auth import HTTPBasicAuth
from tcp import client
import tcp

### Host Definition

In [ ]:
host="https://unstable.thecrossproduct.xyz/v1"

### Connection information

In [ ]:
email=""
password = ""

# 1 - Authentification

## a. User Login

User login is done by client object instanciation:

In [ ]:
api=client(host=host,usermail=email,passwd=password)

You can now retrieve your token as follows:

In [ ]:
token=api.token
print(token)

TIPS: Once you retrieved your token, you can add the token to the env variable "TCP_API_TOKEN". 
After that, you won't need to provide email and password to create a connection

## b. Get user's informations

This endpoints provide user informations such as id, mail, creation date, associated groups...

In [ ]:
user_info=api.query().auth.get()
print(user_info)

## c. Get token's informations

This endpoint verifies the validity of your token and provides some information about it, including token id, user id, expiration date, eventual activation limits, number of uses, scope authorized ...

In [ ]:
print(api.query().auth.validate_auth_token.get())


## d. User logout

In [ ]:
api.query().auth.logout.get()

# 2 - Manage your licences

## a. Get licence information

This endpoints provides a list of licenses associated with your account.

In [ ]:
print(api.query().auth.validate_auth_token.get())

# 3 - Manage your data

This section, allows you to check your data, upload new data from your local repository to the S3 storage or, on the contrary, download data from S3 storage to your local repository.

In order to make it user friendly, we created the functions upload() and download().  

## a. Upload Data

In [ ]:
# Arguments definition
s3_target_uri="test_upload"
local_file_path=""
file_size=str(os.path.getsize(local_file_path))

In [ ]:
#returns True if successfull
print(api.upload(file_path,s3_target_uri))

## b. Download data

In [ ]:
# Arguments definition
target_local_path=""

In [ ]:
#returns True if successfull
print(api.download(s3_target_uri,target_local_path))

## c. List your TCP data

In [ ]:
print(api.query().data.get())

## List TCP Solutions

In [ ]:
print(api.query().app.get())

## Specific informations for Domain

In [ ]:
print(api.query().app.info.get(Domain="railway",App="classification").decode('utf8'))

### List Available remotes

In [ ]:
print(api.query().app.remote.scw.get())

## Run TCP Solution

In [ ]:
#définition des inputs
clouds=["demo/cloud1.laz"]
trj=["demo/trajectory.dxf"]
output="demo/classif"

In [ ]:

print(api.query().app.run.post({"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]},Domain="railway",App="classification"))

In [ ]:
url = host+"v1/app/run?Domain=Railway&App=Classification"
payload = json.dumps({"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"] })
headers = {'Content-Type': 'application/json','x-access-token': token}

mod = requests.request("POST", url, headers=headers, data=payload)

if mod.status_code==200:
    print(mod.text)
else:
    print(mod)

## Check Process status & Download results

In [ ]:
id_process = mod.json()['id']

url = host+"/v1/app/status?Process={}".format(id_process)

headers = {
  'x-access-token': token
}

download_result = requests.request("GET", url, headers=headers)

print(download_result.text)
#le paramètre "output" contiendra un url vers le résultat à télécharger lorsque le traitement sera terminé

## List Processes

In [ ]:
print(api.query().app.list.Process.get())